In [135]:
import pandas as pd
import numpy as np
import re
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [136]:
df = pd.read_csv("/Users/mikegu/Desktop/Big data term project new/bigdata dataset/movies_metadata.csv")
df.head()

/var/folders/3r/stx0z6fn3y7gn9nz8mwg1xfr0000gn/T/ipykernel_56090/367383368.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mikegu/Desktop/Big data term project new/bigdata dataset/movies_metadata.csv")


adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [137]:
movie_id = df['id'].values.tolist()
movie_title = df['original_title'].values.tolist()
movie_overview = df['overview'].values.tolist()
movie_clean = {'id':movie_id,'title':movie_title,'overview':movie_overview}
final = pd.DataFrame.from_dict(movie_clean)
final


id                        title  \
0         862                    Toy Story   
1        8844                      Jumanji   
2       15602             Grumpier Old Men   
3       31357            Waiting to Exhale   
4       11862  Father of the Bride Part II   
...       ...                          ...   
45461  439050                      رگ خواب   
45462  111109          Siglo ng Pagluluwal   
45463   67758                     Betrayal   
45464  227506          Satana likuyushchiy   
45465  461257                     Queerama   

                                                overview  
0      Led by Woody, Andy's toys live happily in his ...  
1      When siblings Judy and Peter discover an encha...  
2      A family wedding reignites the ancient feud be...  
3      Cheated on, mistreated and stepped on, the wom...  
4      Just when George Banks has recovered from his ...  
...                                                  ...  
45461        Rising and falling between a man and woman.  
45462  An artist struggles to finish his work while a...  
45463  When one of her hits goes wrong, a professiona...  
45464  In a small town live two brothers, one a minis...  
45465  50 years after decriminalisation of homosexual...  

[45466 rows x 3 columns]

In [138]:
col=final['overview']

In [139]:
over = col.values.tolist()
over[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [142]:
for i in range(int(len(over))):
    f = open("overviewdataset/"+str(i)+".txt","w")
    f.writelines(str(over[i]))


In [143]:
stopwords = []
corpus=[]
movie_tfidfvector=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,45465):
    with open("overviewdataset/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    corpus.append(text_words)

In [144]:
def read_corpus(corpus):
    for i, line in enumerate(corpus):
        yield gensim.models.doc2vec.TaggedDocument(line, [i])
train_corpus = list(read_corpus(corpus))

In [145]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
movie_vector=[]
for i in range(0,1470):
    movie_vector.append(model.infer_vector(corpus[i]))
movie_vector

[array([ 0.5851769 ,  0.67378575,  0.01236795,  0.43353474, -0.57742465,
         0.4725342 , -0.29453588, -0.05479597, -0.27698696, -0.7749239 ,
        -0.5224522 ,  0.11892994, -0.7211447 ,  0.4269303 ,  0.20884532,
        -0.1512631 ,  0.3352808 , -0.36194953, -0.29357773, -0.21676815,
        -0.98153234, -0.29285425,  1.1007576 , -1.0915232 ,  0.00519537,
         0.27190584,  1.3198905 , -0.9679328 , -0.9099084 , -0.5009795 ,
         0.8281946 , -1.270265  ,  0.2559035 , -0.7865381 , -0.93383473,
         0.38753766,  0.4121129 ,  0.19285367,  0.8032882 , -0.12206892,
         0.18435353, -0.57779807, -1.2403811 ,  0.00270428, -0.13128288,
        -0.14144635, -0.66729915,  0.3250473 , -0.8487324 , -0.36376575],
       dtype=float32),
 array([ 0.39167807,  1.3885489 , -1.0241032 , -0.63761616,  0.00209355,
        -0.4122022 ,  0.5583462 , -0.13050488, -1.0860223 , -0.73252606,
        -0.45477304, -1.0297942 ,  0.5981362 , -0.3509794 , -0.40002927,
         1.7284476 ,  0.227

In [146]:
indices=pd.Series(final.index,index=final.title)
def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

In [147]:
indices['Nothing to Lose']

title
Nothing to Lose     849
Nothing to Lose    6896
dtype: int64

In [148]:
def recommend(movie):
    vector=model.infer_vector(corpus[indices[movie]])
    sims = model.docvecs.most_similar([vector],topn=11)
    sims = sims[1:]
    result=[]
    for i in sims:
        series=final.title.iloc[i[0]]
        result.append(series)
        result.append(i[1])
    re = np.array(result)
    re = re.reshape((10,2))
    return re

In [152]:
recommend('Popstar: Never Stop Never Stopping')

/var/folders/3r/stx0z6fn3y7gn9nz8mwg1xfr0000gn/T/ipykernel_56090/1088360842.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector],topn=11)


array([['Elle: A Modern Cinderella Tale', '0.6333139538764954'],
       ['Begin Again', '0.6272032260894775'],
       ['1 - నేనొక్కడినే', '0.6259869933128357'],
       ['Detroit Rock City', '0.6237151622772217'],
       ['Ride or Die', '0.6144478917121887'],
       ['Pleased to Meet Me', '0.6065956950187683'],
       ['Super Fly', '0.6049697995185852'],
       ['Buying the Cow', '0.6003759503364563'],
       ['Don’t Stop Believin’: Everyman’s Journey', '0.6000251173973083'],
       ['Boogie Nights', '0.5998827219009399']], dtype='<U40')